In [2]:
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import year

from delta import *
import pyspark
from pyspark.sql import SparkSession

delta_spark_version = "2.1.0"
spark_jars_packages = f"io.delta:delta-core_2.12:{delta_spark_version}"

spark = (
            SparkSession.builder.master("local[*]")
                .appName("PySparkLocal")
                .config("spark.executor.memory", "4g")
                .config("spark.driver.memory", "2g")
                .config("spark.memory.offHeap.enabled", True)
                .config("spark.memory.offHeap.size","8g")
                .config("spark.jars.packages", spark_jars_packages)
                .config("spark.sql.sources.partitionOverwriteMode", "dynamic")
                .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
                .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
                .config("spark.databricks.delta.schema.autoMerge.enabled", "true")
                .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")
                .config("spark.databricks.delta.optimize.repartition.enabled", "true")
                .getOrCreate()
        )

:: loading settings :: url = jar:file:/Users/pii/Library/Caches/pypoetry/virtualenvs/delta-demo-1o0m9wMe-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/pii/.ivy2/cache
The jars for the packages stored in: /Users/pii/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0628a22b-c3c4-4c14-8f6e-75e268564e41;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 197ms :: artifacts dl 20ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicte

22/10/11 14:03:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/11 14:03:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/11 14:03:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [31]:
deltaDf = spark.read.format("delta").load('Deltalake/crypto')
deltaDf.count()

5471089

In [32]:
deltaDf.select('isoString').distinct().collect()

[Row(isoString='2022-10-09'), Row(isoString='2022-10-08')]